# Quality Detector

In this script I will create toy CNN that is supposed to detect whether the input image is of a good quality or has artefacts.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gc

## Data Preprocessing

I will start by loading two of the images in. Then I will select from the originals each only one. Aftwards, I will select the 500 images in good and bad quality from the two image and create the classification label for it.

In [2]:
P3_X = np.load('/home/cloud/MRT_Data/unziped/out/P3_X.npy')
P3_Y = np.load('/home/cloud/MRT_Data/unziped/out/P3_Y.npy')
P14_X = np.load('/home/cloud/MRT_Data/unziped/out/P14_X.npy')
P14_Y = np.load('/home/cloud/MRT_Data/unziped/out/P14_Y.npy')

#### Select images from Y only once
To match the number of distorted images in X, the images in Y are each repeated five times subsequently. To get unique images I will only select the first once.

In [3]:
P3_Y = P3_Y[[i for i in range(0,P3_Y.shape[0],5)],:,:,:]

In [4]:
P14_Y = P14_Y[[i for i in range(0,P14_Y.shape[0],5)],:,:,:]

For the distorted pictures saved in P3_X and P14_X I will select on of the five distorted images randomly.

In [5]:
P3_X = P3_X[[i + np.random.randint(0,4) for i in range(0,P3_X.shape[0],5)], :, :,:]

In [6]:
P14_X = P14_X[[i + np.random.randint(0,4) for i in range(0,P14_X.shape[0],5)], :, :,:]

Combine all images and create class vector

In [7]:
shape_x = (4 * P3_X.shape[0],) + P3_X.shape[1:4]

In [8]:
X = np.empty(shape_x,dtype='uint32')
X[0:811,:,:,:] = P3_Y
X[811:1622,:,:,:] = P14_Y
X[1622:2433,:,:,:] = P3_X
X[2433:3244,:,:,:] = P14_X

 Normalize data

In [9]:
X = X / 255

In [10]:
X = X.astype('float32')

In [11]:
Y = np.array([1] * 1622 + [0] * 1622 , dtype='int32')

Shuffle both arrays randomly!

In [12]:
shuffle_index = np.random.choice(range(0,Y.shape[0]), size = Y.shape[0], replace = False)

In [13]:
X = X[shuffle_index,:,:,:]

In [14]:
Y = Y[shuffle_index,]

clean up the mess

In [15]:
del P3_X, P3_Y, P14_X, P14_Y

In [16]:
gc.collect()

23

# Create train and test and validation set

In [17]:
X_train = X[1:2270, :,:,:]
X_eval = X[2270:3244,:,:,:]

Y_train = Y[1:2270,]
Y_eval = Y[2270:3244,]

# Create and train CNN

Create the CNN model

In [31]:
def cnn_model_fn(features, labels, mode):
    
    # Input Layer
    input_layer = features['x']
    print(input_layer.shape)
    
    # Convolutional layer #1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 64,
        kernel_size = [5, 5],
        padding = "same",
        activation= tf.nn.relu)
    
    # Pooling layer 1
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size=[2,2], strides = 2)
    
    # Convolutional layer #2
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 128,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu)
    
    # Pool layer 2 
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2], strides = 2)
    
    # Dense Layer
    pool2flat = tf.reshape(pool2, [-1, 64 *  64 * 128])
    dense = tf.layers.dense(inputs = pool2flat, units = 1024, activation = tf.nn.relu)
    dropout = tf.layers.dropout(inputs = dense, rate = 0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logit layer
    logits = tf.layers.dense(inputs = dropout, units = 2)
    
  
    # Calculate predictions
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)
    
    # Calculate Loss (for both Train and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)

    # Configure the Training OP (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(loss = loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
        
    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    

Load train and test data

In [20]:
train_data = X_train
train_labels = Y_train
eval_data = X_eval
eval_labels = Y_eval

Create the estimator

In [32]:
noise_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/noise_classifier")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/noise_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0a38285ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Set Up a Logging Hook

In [33]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

Train the model

In [34]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [ ]:
noise_classifier.train(
    input_fn=train_input_fn,
    steps=100)

INFO:tensorflow:Calling model_fn.
(100, 256, 256, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/noise_classifier/model.ckpt.
INFO:tensorflow:loss = 0.6913327, step = 1
INFO:tensorflow:Saving checkpoints for 9 into /tmp/noise_classifier/model.ckpt.
INFO:tensorflow:Saving checkpoints for 18 into /tmp/noise_classifier/model.ckpt.
